In [ ]:
import numpy as np
import pickle
import matplotlib.pyplot as plt
import pandas as pd

import torch
import torch.nn.functional as F
from torch.utils.data import Dataset
import torch.optim as optim
from torch.utils.data import DataLoader
import torch.nn as nn

import nibabel as nib
import os
import time
import gc
import copy
import cv2
from sklearn.model_selection import train_test_split
import wandb
from torch.optim.lr_scheduler import CyclicLR
import segmentation_models_pytorch as smp
from segmentation_models_pytorch.encoders import get_preprocessing_fn

import albumentations as A
from albumentations.pytorch import ToTensorV2
from pytorch_toolbelt import losses as L


In [ ]:
device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
#train_mask1,train_mask2,train_mask3,train_mask_all
class train_dl(torch.utils.data.Dataset):
    def __init__(self, frame):
        self.frame =frame
        self.image_files = [f for f in frame['train_images'].tolist()]
        self.mask_files1 = [f for f in frame['train_mask1'].tolist()]
        self.mask_files2 = [f for f in frame['train_mask2'].tolist()]
        self.mask_files3 = [f for f in frame['train_mask3'].tolist()]


    def __getitem__(self, index):
        img,mask=img_trf_train(str(self.image_files[index]),str(self.mask_files1[index]),str(self.mask_files2[index]),str(self.mask_files3[index]))
        return img, mask

    def __len__(self):
        return len(self.frame)
        
class val_dl(torch.utils.data.Dataset):
    def __init__(self,frame):
        self.frame =frame
        self.image_files = [f for f in frame['train_images'].tolist()]
        self.mask_files1 = [f for f in frame['train_mask1'].tolist()]
        self.mask_files2 = [f for f in frame['train_mask2'].tolist()]
        self.mask_files3 = [f for f in frame['train_mask3'].tolist()]

    def __getitem__(self, index):
        img,mask=img_trf_val(str(self.image_files[index]),str(self.mask_files1[index]),str(self.mask_files2[index]),str(self.mask_files3[index]))
        return img, mask

    def __len__(self):
        return len(self.frame)


class test_dl(torch.utils.data.Dataset):
    def __init__(self,frame):
        self.frame =frame
        self.image_files = [f for f in frame['test_images'].tolist()]
        self.mask_files1 = [f for f in frame['test_mask1'].tolist()]
        self.mask_files2 = [f for f in frame['test_mask2'].tolist()]
        self.mask_files3 = [f for f in frame['test_mask3'].tolist()]

    def __getitem__(self, index):
        img,mask=img_trf_val(str(self.image_files[index]),str(self.mask_files1[index]),str(self.mask_files2[index]),str(self.mask_files3[index]))
        return img, mask

    def __len__(self):
        return len(self.frame)


In [ ]:
def img_trf_train(img_path,mask_path1,mask_path2,mask_path3):

    img=cv2.imread(str(img_path), cv2.IMREAD_GRAYSCALE)
    msk1=cv2.imread(str(mask_path1), cv2.IMREAD_GRAYSCALE)
    msk2=cv2.imread(str(mask_path2), cv2.IMREAD_GRAYSCALE)
    msk3=cv2.imread(str(mask_path3), cv2.IMREAD_GRAYSCALE)

    #edged = cv2.Canny(msk, 50, 200) #for edge
    #msk=(edged > 0).astype(int) #for edge
    max_side1=max(img.shape[0],img.shape[1])
    max_side2=max(msk1.shape[0],msk1.shape[1])
    max_side=max(max_side1,max_side2)
    aug = A.Compose([
        A.PadIfNeeded(min_height=max_side, min_width=max_side,border_mode=cv2.BORDER_CONSTANT,value=0,p=1),
        A.Affine(scale=(0.8,1.1), shear=(-20,20), rotate=(-20,20), p=0.8,fit_output=True,keep_ratio=True,mode=0,cval=0),
        #A.GaussianBlur(blur_limit=(3, 7), sigma_limit=0, p=0.1),
        #A.RandomBrightnessContrast(p=0.5),
        A.Resize(128,128),
    ],is_check_shapes=False)
    
    masks=np.stack([msk1,msk2,msk3],axis=0).T
    augmented = aug(image=img, mask=masks)
    
    image_padded = torch.tensor(augmented['image'].T)
    mask_padded = torch.tensor(augmented['mask'].T)
    return image_padded, mask_padded


def img_trf_val(img_path,mask_path1,mask_path2,mask_path3):

    img=cv2.imread(str(img_path), cv2.IMREAD_GRAYSCALE)
    msk1=cv2.imread(str(mask_path1), cv2.IMREAD_GRAYSCALE)
    msk2=cv2.imread(str(mask_path2), cv2.IMREAD_GRAYSCALE)
    msk3=cv2.imread(str(mask_path3), cv2.IMREAD_GRAYSCALE)

    #edged = cv2.Canny(msk, 50, 200) #for edge
    #msk=(edged > 0).astype(int) #for edge
    max_side1=max(img.shape[0],img.shape[1])
    max_side2=max(msk1.shape[0],msk1.shape[1])
    max_side=max(max_side1,max_side2)
    aug = A.Compose([
        A.PadIfNeeded(min_height=max_side, min_width=max_side,border_mode=cv2.BORDER_CONSTANT,value=0,p=1),
        A.Resize(128,128),
    ],is_check_shapes=False)
    
    masks=np.stack([msk1,msk2,msk3],axis=0).T

    augmented = aug(image=img, mask=masks)
    
    image_padded = torch.tensor(augmented['image'].T)
    mask_padded = torch.tensor(augmented['mask'].T)
    return image_padded, mask_padded


In [ ]:
train_data=pd.read_csv('train_data.csv')
test_data=pd.read_csv('test_data.csv')
train_df, val_df = train_test_split(train_data, test_size=0.2, random_state=42)


In [ ]:
test_dataset = test_dl(test_data)

test_dataloader=DataLoader(test_dataset, batch_size=1,pin_memory=True,shuffle=False,num_workers=2)
#test_features,test_labels = next(iter(test_dataloader))

#test_set=torch.utils.data.DataLoader(torch.utils.data.TensorDataset(train_features,train_labels), batch_size=4,pin_memory=True,shuffle=True,num_workers=2)


In [ ]:
import segmentation_models_pytorch as smp
from segmentation_models_pytorch.encoders import get_preprocessing_fn


model = smp.Unet(
    encoder_name="resnet34",        # choose encoder, e.g. mobilenet_v2 or efficientnet-b7
    encoder_weights="imagenet",     # use `imagenet` pre-trained weights for encoder initialization
    in_channels=1,                  # model input channels (1 for gray-scale images, 3 for RGB, etc.)
    classes=3,                      # model output channels (number of classes in your dataset)
)

#preprocess_input = get_preprocessing_fn('resnet34', pretrained='imagenet')
loss_function=L.DiceLoss(mode='multilabel')#binary vagy multiclass
lr=1e-3
optimizer=optim.Adam(model.parameters(), lr=lr, weight_decay=0.000001)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min',factor=0.3, patience=3, verbose=True)
model_name='./Unet_Res34_Imgnet_Dice_all_mask.pt'
model.load_state_dict(torch.load(model_name, weights_only=True, map_location=torch.device('cuda')))
model.to(device)


In [ ]:
from torchvision.utils import save_image

model.eval()
i=0
for images, labels in test_dataloader:
    images=images.unsqueeze(1)
    images=images.to(device=device, dtype=torch.float) 
    outputs = model(images)
    outputs=outputs.squeeze(0)
    print(outputs.shape)
    #save_image(outputs, 'o.png') save full batch
    cv2.imwrite(f'output/unet/{i}.png', outputs.cpu().detach().numpy().T)
    i+=1

In [ ]:
import segmentation_models_pytorch as smp
from segmentation_models_pytorch.encoders import get_preprocessing_fn


model= smp.MAnet(
    encoder_depth=4,                  # Use 4 encoder stages
    decoder_channels=[256, 128, 64,32],  # Decoder channels (adjusted for depth=4)
    in_channels=1,                  # model input channels (1 for gray-scale images, 3 for RGB, etc.)
    classes=3,                        # Predict 3 classes
)

loss_function=L.DiceLoss(mode='multilabel')#binary vagy multiclass
lr=1e-3
optimizer=optim.Adam(model.parameters(), lr=lr, weight_decay=0.000001)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min',factor=0.3, patience=3, verbose=True)
model_name='./MAnet_Res34_Imgnet_Dice_all_mask.pt'
model.load_state_dict(torch.load(model_name, weights_only=True, map_location=torch.device('cuda')))
model.to(device)


In [ ]:
from torchvision.utils import save_image

model.eval()
i=0
for images, labels in test_dataloader:
    images=images.unsqueeze(1)
    images=images.to(device=device, dtype=torch.float) 
    outputs = model(images)
    outputs=outputs.squeeze(0)
    print(outputs.shape)
    #save_image(outputs, 'o.png') save full batch
    cv2.imwrite(f'output/manet/{i}.png', outputs.cpu().detach().numpy().T)
    i+=1

In [ ]:
import segmentation_models_pytorch as smp
from segmentation_models_pytorch.encoders import get_preprocessing_fn


model = smp.DeepLabV3(
    encoder_name="resnet34",           # Encoder backbone
    encoder_depth=4,                  # Use 4 encoder stages
    encoder_weights="imagenet",       # Pre-trained on ImageNet
    in_channels=1,                  # model input channels (1 for gray-scale images, 3 for RGB, etc.)
    classes=3,                        # Predict 3 classes
)

loss_function=L.DiceLoss(mode='multilabel')#binary vagy multiclass
lr=1e-3
optimizer=optim.Adam(model.parameters(), lr=lr, weight_decay=0.000001)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min',factor=0.3, patience=3, verbose=True)
model_name='./deeplabv3_Res34_Imgnet_Dice_all_mask.pt'
model.load_state_dict(torch.load(model_name, weights_only=True, map_location=torch.device('cuda')))
model.to(device)


In [ ]:
from torchvision.utils import save_image

model.eval()
i=0
for images, labels in test_dataloader:
    images=images.unsqueeze(1)
    images=images.to(device=device, dtype=torch.float) 
    outputs = model(images)
    outputs=outputs.squeeze(0)
    print(outputs.shape)
    #save_image(outputs, 'o.png') save full batch    
    cv2.imwrite(f'output/deeplab/{i}.png', outputs.cpu().detach().numpy().T)
    i+=1


In [ ]:

model = smp.Linknet(
    encoder_name="resnet34",           # Encoder backbone
    encoder_depth=4,                  # Use 4 encoder stages
    encoder_weights="imagenet",       # Pre-trained on ImageNet
    in_channels=1,                  # model input channels (1 for gray-scale images, 3 for RGB, etc.)
    classes=3,                        # Predict 3 classes
)

loss_function=L.DiceLoss(mode='multilabel')#binary vagy multiclass
lr=1e-3
optimizer=optim.Adam(model.parameters(), lr=lr, weight_decay=0.000001)
#scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min',factor=0.3, patience=3, verbose=True)
scheduler = CyclicLR(optimizer, 
                 base_lr = 0.00006, # Initial learning rate which is the lower boundary in the cycle for each parameter group
                 max_lr = 1e-3, # Upper learning rate boundaries in the cycle for each parameter group
                 step_size_up = 10, # Number of training iterations in the increasing half of a cycle
                 mode = "exp_range")
model_name='./Linknet_Res34_Imgnet_Dice_all_mask.pt'
model.load_state_dict(torch.load(model_name, weights_only=True, map_location=torch.device('cuda')))
model.to(device)


In [ ]:
from torchvision.utils import save_image

model.eval()
i=0
for images, labels in test_dataloader:
    images=images.unsqueeze(1)
    images=images.to(device=device, dtype=torch.float) 
    labels=labels.to(device=device, dtype=torch.float) 
    outputs = model(images)
    labels=labels.squeeze(0)
    outputs=outputs.squeeze(0)
    print(outputs)
    #save_image(outputs, 'o.png') save full batch
    #plt.imshow(outputs.cpu().detach().numpy().T)
    #plt.show()
    #print(outputs.cpu().detach().numpy().T.max())
    cv2.imwrite(f'output/linknet/{i}.png', outputs.cpu().detach().numpy().T)
    #cv2.imwrite(f'output/orig/{i}.png', labels.cpu().detach().numpy().T)
    i+=1
    if i>1:
        break

In [ ]:
plt.imshow(labels.cpu().detach().numpy().T)

In [ ]:
plt.imshow(outputs.cpu().detach().numpy().T)

In [ ]:
def img_trf_val(img_path,mask_path1,mask_path2,mask_path3):

    img=cv2.imread(str(img_path), cv2.IMREAD_GRAYSCALE)
    msk1=cv2.imread(str(mask_path1), cv2.IMREAD_GRAYSCALE)
    msk2=cv2.imread(str(mask_path2), cv2.IMREAD_GRAYSCALE)
    msk3=cv2.imread(str(mask_path3), cv2.IMREAD_GRAYSCALE)

    #edged = cv2.Canny(msk, 50, 200) #for edge
    #msk=(edged > 0).astype(int) #for edge
    max_side1=max(img.shape[0],img.shape[1])
    max_side2=max(msk1.shape[0],msk1.shape[1])
    max_side=max(max_side1,max_side2)
    aug = A.Compose([
        A.PadIfNeeded(min_height=max_side, min_width=max_side,border_mode=cv2.BORDER_CONSTANT,value=0,p=1),
        A.Resize(128,128),
    ],is_check_shapes=False)
    
    masks=np.stack([msk1,msk2,msk3],axis=0).T

    augmented = aug(image=img, mask=masks)
    
    image_padded = torch.tensor(augmented['image'].T)
    mask_padded = torch.tensor(augmented['mask'].T)
    return image_padded, mask_padded
